In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Download training data from open datasets
training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets
test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:02<00:00, 11028748.42it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 175287.77it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3218200.80it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5048463.17it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batch_size=64

# Create data loaders.
train_dataloader=DataLoader(training_data, batch_size=batch_size)
test_dataloader=DataLoader(test_data, batch_size=batch_size)

for X,y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

# N：batch_size，即一次训练或者推断使用的样本数量
# C：通道数，即图像中每个像素点的特征数
# H：图像的高度
# W：图像的宽度


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### Creating Models

In [ ]:
# Get cpu,gpu or mps device for training.
device=(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten=nn.Flatten()  # 将输入数据展平为一维张量
    self.linear_relu_stack=nn.Sequential(  # 定义了一个包含三个线性层以及ReLU激活函数的神经网络
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  def forward(self,x):
    x=self.flatten(x)
    logits=self.linear_relu_stack(x)  # 得到输出张量logits
    return logits

model=NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimizing the Model Parameters

In [ ]:
# define a loss function and an optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size=len(dataloader.dataset)
  model.train()
  for batch,(X,y) in enumerate(dataloader):
    X,y=X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型的权重参数，使其朝着最小化损失的方向优化
    optimizer.zero_grad()  # 清除梯度，为下一轮迭代做准备

    if batch%100==0:
      loss, current = loss.item(), (batch+1)*len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model ,loss_fn):
  size=len(dataloader.dataset)
  num_batches=len(dataloader)
  model.eval()
  test_loss,correct=0,0
  with torch.no_grad():
    for X,y in dataloader:
      X,y=X.to(device), y.to(device)
      pred=model(X)
      test_loss+=loss_fn(pred,y).item()
      correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
  test_loss/=num_batches
  correct/=size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs=5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader,model,loss_fn,optimizer)
  test(test_dataloader,model,loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298794  [   64/60000]
loss: 2.289197  [ 6464/60000]
loss: 2.265476  [12864/60000]
loss: 2.269550  [19264/60000]
loss: 2.250633  [25664/60000]
loss: 2.212823  [32064/60000]
loss: 2.236142  [38464/60000]
loss: 2.194229  [44864/60000]
loss: 2.201200  [51264/60000]
loss: 2.160766  [57664/60000]
Test Error: 
 Accuracy: 38.9%, Avg loss: 2.157762 

Epoch 2
-------------------------------
loss: 2.172973  [   64/60000]
loss: 2.162876  [ 6464/60000]
loss: 2.098361  [12864/60000]
loss: 2.118320  [19264/60000]
loss: 2.068827  [25664/60000]
loss: 2.003073  [32064/60000]
loss: 2.038586  [38464/60000]
loss: 1.959701  [44864/60000]
loss: 1.967056  [51264/60000]
loss: 1.881310  [57664/60000]
Test Error: 
 Accuracy: 56.7%, Avg loss: 1.886841 

Epoch 3
-------------------------------
loss: 1.926648  [   64/60000]
loss: 1.895735  [ 6464/60000]
loss: 1.769338  [12864/60000]
loss: 1.812011  [19264/60000]
loss: 1.702200  [25664/60000]
loss: 1.648376  [32064/600

### Saving Models

In [13]:
torch.save(model.state_dict(),"model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### Loading Models

In [14]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x,y=test_data[0][0],test_data[0][1]
with torch.no_grad():
  x=x.to(device)
  pred=model(x)
  predicted,actual=classes[pred[0].argmax(0)],classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
